# Getting started

In [25]:
# Standard library imports
import configparser

# Third-party imports
import numpy as np
import pandas as pd
from flask import Flask
import openai
from openai import OpenAI

# Pandas configuration
pd.set_option('display.max_colwidth', None)

## 1. Data

In [2]:
data_sample = pd.read_csv("../data/twitter_data_clean_sample.csv")

In [3]:
data_sample.head()

,customer_tweet,company_tweet,company
0,Ordered this around 2am Friday morning and it made it here already... good job @115830 https://t.co/XXMuII3QwQ,@383517 I am very happy to hear this Pablo:) I hope you enjoy your order.^GA,AmazonHelp
1,"@AmazonHelp what does ""ships in 1-3 weeks"" actually mean? Do you have the item I want in stock or not? Items like this have given me issues","@274096 If your item will ship in 1-3 weeks, this means the item is not in stock and needs to be ordered from our distributor. More info here: https://t.co/V7JYyWd9JF ^RA",AmazonHelp
2,@115821 // Email from Representative not correct. There was someone to receive package. Whoever said different at @118706 is lying.,@528375 I'm sorry you haven't received your package. We'd like to help. Please contact us here: https://t.co/hApLpMlfHN ^AY,AmazonHelp
3,je l’ai déjà l’application amazon jdevrais être immunisé de vos pubs de merde @115821,"@792999 3/3 Ensuite décochez à nouveau les cases que vous aviez sélectionnées. N'oubliez pas de ""Valider"" pour effectuer vos modifications. \nNous espérons que ces informations vous seront utiles.",AmazonHelp
4,"I must say @115830, a package left under a doormat which is full of holes, in the middle of a downpour, is not the best idea #wetelectronics","@776873 I apologize for how your delivery was handled, that is not the experience we want our customers to have. Which courier was assigned delivery of that package, as shown in the order details here: https://t.co/aaDyEz1VgE ^CH",AmazonHelp


In [4]:
data_sample.company.value_counts()

company
AmazonHelp      100
AppleSupport    100
SpotifyCares    100
Uber_Support    100
Name: count, dtype: int64

## 2. How to use the OpenAI API

Create a `config.ini` file containing your OpenAI API credentials

    
    [OPENAI_API]
    OPENAI_KEY = key
    

In [11]:
# Loading OpenAI API key from configuration file
config = configparser.ConfigParser()
config.read('../config.ini') #Path to your configuration file
OPENAI_KEY = config.get('OPENAI_API', 'OPENAI_KEY')

In [12]:
client = OpenAI(api_key=OPENAI_KEY)

### ChatCompletion : Get GPT response to your prompt

#### Documentation : https://platform.openai.com/docs/guides/text-generation/chat-completions-api

In [13]:
message = "Placed an order for some headphones early Monday morning, and they've arrived in 2 days... impressive service!"
company = "Amazon"

print(f"Tweet: {message} \nCompany: {company}")

Tweet: Placed an order for some headphones early Monday morning, and they've arrived in 2 days... impressive service! 
Company: Amazon


In [14]:
instruction =  f"""\
You are a chatbot answering customer's messages. You are working for a company called {company}. Reply to the message below.
#####
Message:
"{message}" """
print(f"Instruction:\n\n{instruction}")

Instruction:

You are a chatbot answering customer's messages. You are working for a company called Amazon. Reply to the message below.
#####
Message:
"Placed an order for some headphones early Monday morning, and they've arrived in 2 days... impressive service!" 


In [16]:
messages = [
    {"role": "user", "content": instruction}
]

response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=messages,
    temperature = 0.7) # temperature ranges from 0 (deterministic) to 1 (creative)

In [17]:
generated_text_vanilla = response.choices[0].message.content
print(f"Answer generated:\n\n{generated_text_vanilla}")

Answer generated:

Thank you for your message! We're thrilled to hear that your headphones arrived so quickly. We strive to provide our customers with the best possible service, and it's great to know we met your expectations. If you have any questions or need further assistance, feel free to reach out. Enjoy your new headphones!


### Embedding model : Get the embedding of a text

#### Documentation : https://platform.openai.com/docs/guides/embeddings

In [18]:
message_1 = "Placed an order for some headphones early Monday morning, and they've arrived in 2 days... impressive service!"
message_2 = "Ordered this around 2am Friday morning and it made it here already... good job @115830 https://t.co/XXMuII3QwQ"
message_3 = "I hate Amazon!!!"

In [20]:
def get_embedding(text, model="text-embedding-3-small"):
   """Get the embedding of an input text"""
   text = text.replace("\n", " ")
   return client.embeddings.create(input = [text], model=model).data[0].embedding

In [21]:
embedding_message_1 = get_embedding(message_1)
embedding_message_2 = get_embedding(message_2)
embedding_message_3 = get_embedding(message_3)

In [22]:
print(f"Message 1 : {message_1}\nEmbedding length : {len(embedding_message_1)}\nEmbedding:\n{embedding_message_1}")

Message 1 : Placed an order for some headphones early Monday morning, and they've arrived in 2 days... impressive service!
Embedding length : 1536
Embedding:
[-0.02072647213935852, -0.03096633590757847, -0.0742698535323143, -0.029683267697691917, 0.02990533784031868, 0.003059621900320053, 0.014878645539283752, 0.055171892046928406, -0.0006924245390109718, -0.02960924431681633, -0.011707988567650318, -0.008420129306614399, -0.03195330873131752, 0.00796982180327177, -0.01904861442744732, 0.013657264411449432, -0.03970106318593025, -0.025686020031571388, -0.06277159601449966, 0.05961327999830246, 0.029683267697691917, -0.0024319677613675594, 0.04895395040512085, 0.025735368952155113, -0.011917720548808575, -0.00942560937255621, -0.01109112985432148, 0.007426985539495945, -0.00762438029050827, -0.041082825511693954, 0.049299392849206924, -0.0169389545917511, 0.03116372972726822, -0.020208310335874557, -0.02468670718371868, -0.0005432217149063945, 0.003913972061127424, 0.011800517328083515,

#### Compare two embeddings to find similarity

A cosine similarity close to 1 implies that the sentence embeddings are very similar, meaning their vectors point in almost the same direction. This suggests the sentences have similar meanings or semantic content.

A cosine similarity around 0 indicates that the sentence embeddings are orthogonal (or near-orthogonal) to each other in the vector space, suggesting that the sentences are unrelated or have neutral similarity.

A cosine similarity close to -1 indicates that the embeddings are diametrically opposed in the vector space, suggesting that the sentences are highly dissimilar or have opposite meanings.

In [24]:
def cosine_similarity(A, B):
    dot_product = np.dot(A, B)
    magnitude_A = np.linalg.norm(A)
    magnitude_B = np.linalg.norm(B)
    return dot_product / (magnitude_A * magnitude_B)

In [26]:
similarity_message1_message2 = cosine_similarity(embedding_message_1, embedding_message_2)
print(f"Message 1 : {message_1}\nMessage 2 : {message_2}\n\nSimilarity: {similarity_message1_message2}")

Message 1 : Placed an order for some headphones early Monday morning, and they've arrived in 2 days... impressive service!
Message 2 : Ordered this around 2am Friday morning and it made it here already... good job @115830 https://t.co/XXMuII3QwQ

Similarity: 0.6050187328723245


In [27]:
similarity_message1_message3 = cosine_similarity(embedding_message_1, embedding_message_3)
print(f"Message 1 : {message_1}\nMessage 3 : {message_3}\n\nSimilarity: {similarity_message1_message3}")

Message 1 : Placed an order for some headphones early Monday morning, and they've arrived in 2 days... impressive service!
Message 3 : I hate Amazon!!!

Similarity: 0.16386384482561167


### Augment your prompt : RAG principle

In [28]:
# Customer's Tweet to answer 
tweet = "Placed an order for some headphones early Monday morning, and they've arrived in 2 days... impressive service!"

In [29]:
# Similar message with the agent's reply

message = data_sample.head(1).customer_tweet[0]
reply = data_sample.head(1).company_tweet[0]
company = data_sample.head(1).company[0]

print(f"Customer's Tweet: {message}\nCompany's Tweet: {reply}\nCompany: {company}")

Customer's Tweet: Ordered this around 2am Friday morning and it made it here already... good job @115830 https://t.co/XXMuII3QwQ
Company's Tweet: @383517 I am very happy to hear this Pablo:) I hope you enjoy your order.^GA
Company: AmazonHelp


In [30]:
instruction =  f"""\
You are a chatbot answering customer's tweet. You are working for a company called Amazon. 
You are provided with an example of a similar interaction between a customer and an agent. Reply to the customer's tweet in the same tone, structure and style than the provided example.

#####
Example :
Customer : "{message}"
Agent : "{reply}"

#####
Tweet:
"{tweet}"
"""
print(f"Instruction:\n\n{instruction}")

Instruction:

You are a chatbot answering customer's tweet. You are working for a company called Amazon. 
You are provided with an example of a similar interaction between a customer and an agent. Reply to the customer's tweet in the same tone, structure and style than the provided example.

#####
Example :
Customer : "Ordered this around 2am Friday morning and it made it here already... good job @115830 https://t.co/XXMuII3QwQ"
Agent : "@383517 I am very happy to hear this Pablo:) I hope you enjoy your order.^GA"

#####
Tweet:
"Placed an order for some headphones early Monday morning, and they've arrived in 2 days... impressive service!"



In [31]:
messages = [
    {"role": "user", "content": instruction}
]

response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=messages,
    temperature = 0.7)

In [32]:
generated_text = response.choices[0].message.content
print(f"Answer generated:\n\n{generated_text}")

Answer generated:

"@CustomerName I'm thrilled to hear your headphones arrived so quickly! 🎧 Enjoy your new sound experience! ^GA"


In [33]:
# Vanilla answer without augmentation

print(f"Vanilla answer without augmentation:\n\n{generated_text_vanilla}")

Vanilla answer without augmentation:

Thank you for your message! We're thrilled to hear that your headphones arrived so quickly. We strive to provide our customers with the best possible service, and it's great to know we met your expectations. If you have any questions or need further assistance, feel free to reach out. Enjoy your new headphones!
